In [10]:
import subprocess

def search_docker_images_by_name(search_term):
    # Using Docker CLI command to list images and then filter by name
    cmd = ["docker", "images", "--filter", f"reference=*{search_term}*", "--format", "{{.Repository}}:{{.Tag}}"]
    matching_images = subprocess.check_output(cmd).decode('utf-8').strip().split('\n')
    
    # Return non-empty image names
    return [image for image in matching_images if image]

# Search for a term
term = "auth-service"
results = search_docker_images_by_name(term)

print(f"\nImages with names containing '{term}':")
for image in results:
    print(image)



Images with names containing 'auth-service':
auth-service:latest


In [22]:
import subprocess

def search_docker_hub(query):
    try:
        result = subprocess.check_output(['docker', 'search', query], universal_newlines=True)
        print(result)
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e.output}")

def search_local_images(query):
    try:
        # Get a list of all local images
        images = subprocess.check_output(['docker', 'images', '--format', '{{.Repository}}:{{.Tag}}'], universal_newlines=True)
        matching_images = [image for image in images.split('\n') if query.lower() in image.lower()]
        
        if matching_images:
            print("\n".join(matching_images))
        else:
            print("No local images matched the query.")
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e.output}")


if __name__ == "__main__":
    keyword = input("Enter search keyword: ")
    choice = input("Search in (1) Docker Hub or (2) Local images? Enter choice number: ").strip()

    if choice == "1":
        search_docker_hub(keyword)
    elif choice == "2":
        search_local_images(keyword)
    else:
        print("Invalid choice.")



NAME                                           DESCRIPTION                                     STARS     OFFICIAL   AUTOMATED
consul                                         Consul is a datacenter runtime that provides…   1428      [OK]       
satosa                                         SATOSA translates between authentication pro…   8         [OK]       
bitnami/service-catalog-archived               A copy of the container images of the deprec…   0                    
redislabs/service-broker                                                                       0                    
vmware/harbor-ui                               Container for Harbor's UI, authentication an…   18                   
redislabs/service-broker-internal                                                              0                    
istio/servicegraph                                                                             4                    
sslhep/servicex_func_adl_uproot_transformer            

In [23]:
import subprocess

def search_docker_hub(username, query):
    try:
       
        full_query = f"{username}/{query}"
        result = subprocess.check_output(['docker', 'search', full_query], universal_newlines=True)
        
        
        lines = result.split('\n')
        print("\n".join(lines[:6]))
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e.output}")

def search_local_images(query):
    try:
        # Get a list of all local images
        images = subprocess.check_output(['docker', 'images', '--format', '{{.Repository}}:{{.Tag}}'], universal_newlines=True)
        matching_images = [image for image in images.split('\n') if query.lower() in image.lower()]
        
        # Print only the first 5 matching images
        print("\n".join(matching_images[:5]))
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e.output}")

if __name__ == "__main__":
    keyword = input("Enter search keyword: ")
    choice = input("Search in (1) Docker Hub or (2) Local images? Enter choice number: ").strip()

    if choice == "1":
        username = input("Enter Docker Hub username to search within: ")
        
        search_docker_hub(username, keyword)
    elif choice == "2":
        search_local_images(keyword)
    else:
        print("Invalid choice.")



NAME                                           DESCRIPTION                                     STARS     OFFICIAL   AUTOMATED
consul                                         Consul is a datacenter runtime that provides…   1428      [OK]       
bitnami/service-catalog-archived               A copy of the container images of the deprec…   0                    
redislabs/service-broker                                                                       0                    
redislabs/service-broker-internal                                                              0                    
istio/servicegraph                                                                             4                    


In [4]:
import requests

def search_docker_hub_api(username, query):
    url = f"https://hub.docker.com/v2/repositories/{username}/?page_size=5"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        if not data['results']:
            print(f"No repositories found for username {username}.")
            return
        
        matching_repos = [repo for repo in data['results'] if query.lower() in repo['name'].lower() or query.lower() in repo['description'].lower()]
        
        if not matching_repos:
            print(f"No matching repositories found for the keyword '{query}'.")
            return

        for repo in matching_repos:
            print(f"Name: {repo['name']}")
            print(f"Description: {repo['description']}")
            print('-' * 40)
    else:
        print(f"Error occurred: {response.status_code} - {response.text}")

if __name__ == "__main__":
    username = input("Enter Docker Hub username to search within: ")
    keyword = input("Enter search keyword: ")
    search_docker_hub_api(username, keyword)



No repositories found for username Redis.


In [27]:
#working only with repo name !!!!!
import subprocess

def search_docker_hub_for_user(username, query):
    try:
        full_query = f"{username}/{query}"
        result = subprocess.check_output(['docker', 'search', full_query], universal_newlines=True)
        
        # Split the results by line and filter lines that start with the username.
        lines = result.split('\n')
        filtered_results = [line for line in lines if line.startswith(username)]
        
        print("\n".join(filtered_results))
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e.output}")

if __name__ == "__main__":
    user_to_search = input("Enter Docker Hub username to search within: ")
    query = input("Enter search keyword: ")
    search_docker_hub_for_user(user_to_search, query)



In [17]:
#final search docker hub
import requests

DOCKER_HUB_API_URL = "https://hub.docker.com/api/content/v1/products/search?sort=updated_at&order=desc&source=community&page_size=50&q="

def search_global_and_filter_by_username(username, query):
    # Constructing the URL
    url = DOCKER_HUB_API_URL + query

    response = requests.get(url)
    response.raise_for_status()

    results = response.json()['summaries']

    # Filter results based on the username
    matching_repos = [repo for repo in results if repo['name'].split('/')[0].strip() == username]

    # Debug: print all repos that match the username
    print("\nRepositories for the username:", username)
    for repo in matching_repos:
        print(repo['name'], "-", repo.get('short_description', ""))

    return matching_repos

if __name__ == "__main__":
    target_username = input("Enter the Docker Hub username to filter by: ")
    query = input("Enter your search query: ")

    repos = search_global_and_filter_by_username(target_username, query)

    if repos:
        for repo in repos:
            print(f"\nName: {repo['name']}\nDescription: {repo.get('short_description', '')}\n{'-' * 80}")
    else:
        print("No repositories found for the given username matching the query.")
#what is the best soultion driven management for network 
#http auth does not work with TortoiseGi
#Unknown and noisy fields when executing kubectl run and outputting to yaml


Repositories for the username: asa7755
asa7755/repo4302 - Multiple imagePullSecrets in a Kubernetes Deployment for the same URL
asa7755/repo4178 - Kubernetes permissions for Helm 3 install to specific namespace
asa7755/repo4142 - Correct YAML format for kubernetes migration

Name: asa7755/repo4302
Description: Multiple imagePullSecrets in a Kubernetes Deployment for the same URL
--------------------------------------------------------------------------------

Name: asa7755/repo4178
Description: Kubernetes permissions for Helm 3 install to specific namespace
--------------------------------------------------------------------------------

Name: asa7755/repo4142
Description: Correct YAML format for kubernetes migration
--------------------------------------------------------------------------------


In [11]:
#Total Questions Answered Correctly
import json
import requests

DOCKER_HUB_API_URL = "https://hub.docker.com/api/content/v1/products/search?sort=updated_at&order=desc&source=community&page_size=5&q="

def search_global_and_filter_by_username(username, query):
    url = DOCKER_HUB_API_URL + query
    response = requests.get(url)
    response.raise_for_status()
    results = response.json().get('summaries', [])
    
    # Check if results is not None and is iterable
    if results is None:
        print(f"Warning: No 'summaries' field for query: {query}.")
        return []

    matching_repos = [repo for repo in results if repo['name'].split('/')[0].strip() == username]
    return matching_repos

def automate_search_and_check():
    with open("/Users/Documents/final-code-microservice-paper/experiments/test_corpus_100.json", "r") as corpus_file:
        test_corpus_100 = json.load(corpus_file)

    with open("/Users/Documents/final-code-microservice-paper/experiments/test_passage_100.json", "r") as passage_file:
        test_passage_100 = json.load(passage_file)

    target_username = "asa7755"
    correct_count = 0
    incorrect_count = 0

    for index, query in test_corpus_100.items():
        #print(f"Searching for query: {query}")
        repos = search_global_and_filter_by_username(target_username, query)
        repo_indices = [int(repo['name'].split('repo')[-1]) for repo in repos]
        correct_answers = test_passage_100[index]
        #is_correct = all(answer in repo_indices for answer in correct_answers)
        is_correct = any(answer in repo_indices for answer in correct_answers)

        if is_correct:
            correct_count += 1
            #print("Correct!\n")
        else:
            incorrect_count += 1
            print(f"Incorrect for query: {query}")
            print(f"Expected: {correct_answers}")
            print(f"Returned: {repo_indices}\n")



    print(f"Total Correct: {correct_count}")
    print(f"Total Incorrect: {incorrect_count}")

if __name__ == "__main__":
    automate_search_and_check()


Incorrect for query: Start new K8s Pod within K8s Pod without "Privileges"
Expected: [531, 3414, 870, 4364, 2546, 2468, 590, 487, 1647, 2126]
Returned: [4743, 4495]

Incorrect for query: Service proxy with affinity based on URL
Expected: [481, 870, 1835, 4372, 2680, 1634, 324, 2002, 547, 3318]
Returned: [5062, 4899]

Incorrect for query: Kubernetes Storage on bare-metal/private cloud
Expected: [1155, 699, 4628, 1024, 543, 1855, 5021, 2335, 3861, 805]
Returned: []

Incorrect for query: Google cloud logging, using python with docker container
Expected: [1106, 728, 1104, 936, 1027, 1936, 1030, 803, 853, 1512]
Returned: []

Incorrect for query: kubernetes is not routing traffic to grpc service
Expected: [2800, 2049, 3405, 5142, 1193, 5047, 2042, 5028, 5385, 1747]
Returned: []

Incorrect for query: Start kubernetes container with specific command
Expected: [281, 625, 729, 2378, 2619, 3163, 3245, 3370, 3502, 3632]
Returned: [5062]

Incorrect for query: Can I get a Pod's pause container ID vi

In [13]:
#Total Correcte Answeres 
import json
import requests

DOCKER_HUB_API_URL = "https://hub.docker.com/api/content/v1/products/search?sort=updated_at&order=desc&source=community&page_size=5&q="

def search_global_and_filter_by_username(username, query):
    url = DOCKER_HUB_API_URL + query
    response = requests.get(url)
    response.raise_for_status()
    results = response.json().get('summaries', [])
    
    # Check if results is not None and is iterable
    if results is None:
        print(f"Warning: No 'summaries' field for query: {query}.")
        return []

    matching_repos = [repo for repo in results if repo['name'].split('/')[0].strip() == username]
    return matching_repos

def automate_search_and_check():
    with open("/Users/Documents/final-code-microservice-paper/experiments/test_corpus_100.json", "r") as corpus_file:
        test_corpus_100 = json.load(corpus_file)

    with open("/Users/Documents/final-code-microservice-paper/experiments/test_passage_100.json", "r") as passage_file:
        test_passage_100 = json.load(passage_file)

    target_username = "asa7755"
    correct_answers_count = 0
    incorrect_count = 0

    for index, query in test_corpus_100.items():
        repos = search_global_and_filter_by_username(target_username, query)
        repo_indices = [int(repo['name'].split('repo')[-1]) for repo in repos]
        correct_answers = test_passage_100[index]

        correct_for_this_query = sum(answer in repo_indices for answer in correct_answers)
        correct_answers_count += correct_for_this_query

        if correct_for_this_query == 0:
            incorrect_count += 1
            print(f"Incorrect for query: {query}")
            print(f"Expected: {correct_answers}")
            print(f"Returned: {repo_indices}\n")

    print(f"Total Correct Answers: {correct_answers_count}")
    print(f"Total Incorrect Queries: {incorrect_count}")

if __name__ == "__main__":
    automate_search_and_check()


Incorrect for query: Start new K8s Pod within K8s Pod without "Privileges"
Expected: [531, 3414, 870, 4364, 2546, 2468, 590, 487, 1647, 2126]
Returned: [4743]

Incorrect for query: Service proxy with affinity based on URL
Expected: [481, 870, 1835, 4372, 2680, 1634, 324, 2002, 547, 3318]
Returned: [5062, 4899]

Incorrect for query: Kubernetes Storage on bare-metal/private cloud
Expected: [1155, 699, 4628, 1024, 543, 1855, 5021, 2335, 3861, 805]
Returned: []

Incorrect for query: Google cloud logging, using python with docker container
Expected: [1106, 728, 1104, 936, 1027, 1936, 1030, 803, 853, 1512]
Returned: []

Incorrect for query: kubernetes is not routing traffic to grpc service
Expected: [2800, 2049, 3405, 5142, 1193, 5047, 2042, 5028, 5385, 1747]
Returned: []

Incorrect for query: Start kubernetes container with specific command
Expected: [281, 625, 729, 2378, 2619, 3163, 3245, 3370, 3502, 3632]
Returned: [5062]

Incorrect for query: Can I get a Pod's pause container ID via the 